# Processing dual-pol data

Here, we'll try to process some dual-polarisation data. We want to:
- Remove the noise,
- Correct from attenuation
- Unfold velocity

In [ ]:
import imp
import warnings

import pyart
import numpy as np
import matplotlib.pyplot as plt

warnings.simplefilter('ignore')

In [ ]:
# Loading 2 functions that were too big to be put in that notebook ;-)
cpol_helper = imp.load_source('cpol_helper', './cpol_helper.py')

In [ ]:
file = 'cpol_20060225_122000_cp.nc'
radar = pyart.io.read(file)

In [ ]:
# Radar data is in radar.fields
for field in sorted(radar.fields.keys()):
    print(f"{field} has units {radar.fields[field]['units']}")

Python tip: You can format a string with the value of a variable, e.g.:
```python
a = 2
print(f"The value of a is {a}")
>>>> "The value of a is 2"
```

In [ ]:
display = pyart.graph.RadarDisplay(radar)

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(12, 15))
ax = ax.ravel()  # Ravel, like flatten, turns a multidimensional array into 1D array

display.plot_ppi('reflectivity', ax=ax[0], sweep=0, cmap='pyart_NWSRef')
display.plot_ppi('differential_reflectivity', ax=ax[1], sweep=0)
display.plot_ppi('velocity', ax=ax[2], sweep=0, cmap='pyart_NWSVel', vmin=-13.3, vmax=13.3)
display.plot_ppi('spectrum_width', ax=ax[3], sweep=0)
display.plot_ppi('differential_phase', ax=ax[4], sweep=0)
display.plot_ppi('cross_correlation_ratio', ax=ax[5], sweep=0, vmin=0.5, vmax=1.05)

for myax in ax:
    display.plot_range_rings([50, 100, 150], ax=myax)
    myax.set_aspect(1)  # I like my axes square.
    myax.set_xlim(-150, 150)
    myax.set_ylim(-150, 150)
    
fig.tight_layout()
plt.show()

So the raw data contains plenty of noise, folding on $\phi_{dp}$ and the Doppler velocity, and strong attenuation too. Let's try to remove the noise.

# Noise removal

In [ ]:
# Let's initialize a gatefilter.
gf = pyart.filters.GateFilter(radar)
gf.exclude_outside('differential_reflectivity', -3.0, 7.0)
gf.exclude_outside('reflectivity', -20.0, 80.0)

We have a signal to noise ratio, we want to determine the cutoff value between signal and noise 

In [ ]:
display.plot_ppi('signal_to_noise_ratio')
plt.axes().set_aspect(1)
plt.show()

In [ ]:
# Plot the PDF of the SNR and try to find the cutoff value.
plt.hist(radar.fields['signal_to_noise_ratio']['data'].flatten(), range=[0, 20], bins=20)
plt.xlabel('Signal to noise ratio (dB)')
plt.ylabel('PDF')
plt.show()

In [ ]:
gf.exclude_below('signal_to_noise_ratio', 8)  # 8 is our cutoff value.

In [ ]:
display.plot_ppi('reflectivity', cmap='pyart_NWSRef', gatefilter=gf)
display.plot_range_rings([50, 100, 150])
plt.axes().set_aspect(1)
plt.show()

## Exercise 1
Compute the texture of the differential phase (function `cpol_helper.texture`) and use it to identify and remove the noise, like we did with the signal_to_noise_ratio.

In [ ]:
# You can always find help in: cpol_helper.texture?

In [ ]:
# Uncomment for solution.
# %load cpol_exercise_1.py

In [ ]:
gf = pyart.correct.despeckle_field(radar, 'reflectivity', gatefilter=gf)

In [ ]:
# Let's see how the different fields look like without the noise.
fig, ax = plt.subplots(3, 2, figsize=(12, 15))
ax = ax.ravel()

display.plot_ppi('reflectivity', ax=ax[0], sweep=0, gatefilter=gf, cmap='pyart_NWSRef')
display.plot_ppi('differential_reflectivity', ax=ax[1], sweep=0, gatefilter=gf)
display.plot_ppi('velocity', ax=ax[2], sweep=0, cmap='pyart_NWSVel', vmin=-13.3, vmax=13.3, gatefilter=gf)
display.plot_ppi('spectrum_width', ax=ax[3], sweep=0, gatefilter=gf)
display.plot_ppi('differential_phase', ax=ax[4], sweep=0, gatefilter=gf)
display.plot_ppi('cross_correlation_ratio', ax=ax[5], sweep=0, vmin=0.5, vmax=1.05, gatefilter=gf)

for myax in ax:
    display.plot_range_rings([50, 100, 150], ax=myax)
    myax.set_aspect(1)  # I like my axes square.
    myax.set_xlim(-150, 150)
    myax.set_ylim(-150, 150)
    
fig.tight_layout()
plt.show()

# Correct the effects of the attenuation

To correct from the attenuation we need 2 things: $\phi_{dp}$ and $K_{dp}$. However, the processing and unfolding of $\phi_{dp}$ is relatively complicated and take a certain amount of time (Py-ART does it, cf. function `pyart.correct.phase_proc_lp`). Thankfully, this is something that has been done for this file, look for the `corrected_differential_phase` and the `specific_differential_phase`

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax = ax.ravel()

display.plot_ppi('differential_phase', ax=ax[0], sweep=0, vmin=-360, vmax=360)
display.plot_ppi('corrected_differential_phase', ax=ax[1], sweep=0, vmin=-360, vmax=360)

for myax in ax:
    display.plot_range_rings([50, 100, 150], ax=myax)
    myax.set_aspect(1)  # I like my axes square.
    myax.set_xlim(-150, 150)
    myax.set_ylim(-150, 150)
    
fig.tight_layout()
plt.show()

In [ ]:
display.plot_ppi('specific_differential_phase')
plt.axes().set_aspect(1)
plt.show()

In [ ]:
atten_meta, zh_corr = pyart.correct.calculate_attenuation(radar, 0, ncp_field='cross_correlation_ratio')
zh_corr['data'] = np.ma.masked_where(gf.gate_excluded, zh_corr['data'])
radar.add_field('specific_attenuation_zh', atten_meta)
radar.add_field('corrected_reflectivity', zh_corr)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax = ax.ravel()

display.plot_ppi('reflectivity', ax=ax[0], sweep=0, cmap='pyart_NWSRef')
display.plot_ppi('corrected_reflectivity', ax=ax[1], sweep=0, cmap='pyart_NWSRef')

for myax in ax:
    display.plot_range_rings([50, 100, 150], ax=myax)
    myax.set_aspect(1)  # I like my axes square.
    myax.set_xlim(-150, 150)
    myax.set_ylim(-150, 150)
    
fig.tight_layout()
plt.show()

## Exercise 2

Correct the differential reflectivity field from the attenuation. 

The attenuation on $Z_{dr}$ is defined as (cf. [Bringi et al. 2001](https://ieeexplore-ieee-org.ezproxy.lib.monash.edu.au/abstract/document/951081) ):

$$ A(r, \theta) = \alpha \int_0^R K_{dp}(r, \theta) dr, $$
where $\alpha= 0.016$, and
$$ Z_{dr, corrected} = Z_{dr} + A(r, \theta)$$

In [ ]:
# Uncomment for solution
# %load cpol_exercise_2.py

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax = ax.ravel()

display.plot_ppi('differential_reflectivity', ax=ax[0], sweep=0)
display.plot_ppi('corrected_differential_reflectivity', ax=ax[1], sweep=0, gatefilter=gf)

for myax in ax:
    display.plot_range_rings([50, 100, 150], ax=myax)
    myax.set_aspect(1)  # I like my axes square.
    myax.set_xlim(-150, 150)
    myax.set_ylim(-150, 150)
    
fig.tight_layout()
plt.show()

# Dealias velocity.

In [ ]:
vdop_vel = pyart.correct.dealias_region_based(radar, vel_field='velocity', gatefilter=gf, nyquist_vel=13.3)
vdop_vel['units'] = 'm/s'

In [ ]:
radar.add_field('corrected_velocity', vdop_vel, replace_existing=True)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax = ax.ravel()

display.plot_ppi('velocity', ax=ax[0], sweep=0, cmap='pyart_NWSVel', gatefilter=gf, vmin=-26, vmax=26)
display.plot_ppi('corrected_velocity', ax=ax[1], sweep=0, cmap='pyart_NWSVel', vmin=-26, vmax=26)

for myax in ax:
    display.plot_range_rings([50, 100, 150], ax=myax)
    myax.set_aspect(1)  # I like my axes square.
    myax.set_xlim(-150, 150)
    myax.set_ylim(-150, 150)
    
fig.tight_layout()
plt.show()

# Exercise 3

Plot the corrected velocity field for all elevations and check if it has been properly corrected.

In [ ]:
# Uncomment for solution
# %load cpol_exercise_3.py